# <font color="#418FDE" size="6.5" uppercase>**Types And Identity**</font>

>Last update: 20251220.
    
By the end of this Lecture, you will be able to:
- Use type, isinstance, and issubclass to inspect and reason about object types and class hierarchies. 
- Interpret id values to understand object identity and lifetime in simple scenarios. 
- Apply len to built-in containers and custom objects that implement the length protocol. 


## **1. Type Checks and isinstance**

### **1.1. Exact Type Inspection**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_06/Lecture_A/image_01_01.jpg?v=1766291950" width="250">



>* Identify the object’s precise, concrete class type
>* Distinguish similar objects by their exact blueprint

>* Different record types require different processing paths
>* Exact type checks route records to correct handlers

>* Exact type checks reduce flexibility and extensibility
>* Use them sparingly, except in strict contexts



In [ ]:
#@title Python Code - Exact Type Inspection

# Demonstrate exact type inspection using type with simple numeric examples.
# Show difference between exact type checks and isinstance polymorphic checks.
# Help decide when strict type boundaries are appropriate in simple programs.

# Create three numeric values that look similar but have different concrete types.
int_value = 10
float_value = 10.0
bool_value = True

# Show the exact type for each value using the built in type function.
print("Exact types:", type(int_value), type(float_value), type(bool_value))

# Perform exact type comparisons using type equality for each created value.
print("int_value is exactly int:", type(int_value) is int)
print("float_value is exactly int:", type(float_value) is int)
print("bool_value is exactly int:", type(bool_value) is int)

# Now demonstrate isinstance which considers inheritance relationships between classes.
print("int_value isinstance int:", isinstance(int_value, int))
print("bool_value isinstance int:", isinstance(bool_value, int))

# Show a small custom class hierarchy for configuration style objects.
class BaseConfig:
    pass


class SpecialConfig(BaseConfig):
    pass


base_config = BaseConfig()
special_config = SpecialConfig()

# Exact type inspection distinguishes strictly between base and subclass instances.
print("Exact base_config type is BaseConfig:", type(base_config) is BaseConfig)
print("Exact special_config type is BaseConfig:", type(special_config) is BaseConfig)

# isinstance treats subclass instances as instances of their parent base class.
print("special_config isinstance BaseConfig:", isinstance(special_config, BaseConfig))



### **1.2. isinstance for polymorphism**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_06/Lecture_A/image_01_02.jpg?v=1766291993" width="250">



>* Focus on what objects do, not are
>* Use isinstance to accept related, compatible objects safely

>* isinstance lets code work with general renderers
>* New renderer subclasses plug in without changing logic

>* isinstance unifies related types across modules, libraries
>* Enables safe, general code independent of subclasses



In [ ]:
#@title Python Code - isinstance for polymorphism

# Demonstrate isinstance usage for polymorphic renderer family behavior.
# Show shared base class with multiple specific renderer subclasses.
# Use isinstance checks to safely call shared rendering interface methods.

from abc import ABC, abstractmethod

class ReportRenderer(ABC):
    @abstractmethod
    def render_header(self, title):
        pass

    @abstractmethod
    def render_body(self, content):
        pass

class TextRenderer(ReportRenderer):
    def render_header(self, title):
        return f"TEXT HEADER: {title}"

    def render_body(self, content):
        return f"TEXT BODY: {content}"

class HtmlRenderer(ReportRenderer):
    def render_header(self, title):
        return f"<h1>{title}</h1>"

    def render_body(self, content):
        return f"<p>{content}</p>"

class InchesRenderer(ReportRenderer):
    def render_header(self, title):
        return f"Inches Report: {title}"

    def render_body(self, content):
        return f"Measured length: {content} inches"

class NotARenderer:
    def something_else(self):
        return "I am unrelated here"

def show_report(renderer, title, content):
    if isinstance(renderer, ReportRenderer):
        header = renderer.render_header(title)
        body = renderer.render_body(content)
        print("Using renderer:", type(renderer).__name__)
        print(header)
        print(body)
    else:
        print("Object rejected, not a ReportRenderer instance.")

text_renderer = TextRenderer()
html_renderer = HtmlRenderer()
length_renderer = InchesRenderer()
stranger_object = NotARenderer()

show_report(text_renderer, "Sales Summary", "Total revenue was 500 dollars.")
show_report(html_renderer, "User Count", "Active users reached 1200 today.")
show_report(length_renderer, "Board Length", "96")
show_report(stranger_object, "Ignored", "This should be rejected.")



### **1.3. Prefer isinstance Over type**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_06/Lecture_A/image_01_03.jpg?v=1766292043" width="250">



>* Check if objects fit a broad category
>* This keeps code flexible, reusable, and extensible

>* Use abstractions so new payment types fit
>* General capability checks reduce future code changes

>* Exact type checks freeze designs and breakability
>* Broader categories keep code extensible, testable, adaptable



In [ ]:
#@title Python Code - Prefer isinstance Over type

# Demonstrate why isinstance checks are usually better than exact type checks.
# Show how subclasses automatically work with isinstance based checks.
# Compare behavior of type equality and isinstance with simple payment classes.

class PaymentMethod:
    def process_payment(self, amount_dollars):
        print(f"Processing generic payment of {amount_dollars} dollars.")


class CreditCardPayment(PaymentMethod):
    def process_payment(self, amount_dollars):
        print(f"Charging credit card for {amount_dollars} dollars.")


class GiftCardPayment(PaymentMethod):
    def process_payment(self, amount_dollars):
        print(f"Using gift card for {amount_dollars} dollars.")


def handle_payment_strict(payment, amount_dollars):
    if type(payment) is PaymentMethod:
        print("Strict handler accepts this payment object.")
        payment.process_payment(amount_dollars)
    else:
        print("Strict handler rejects this payment object.")


def handle_payment_flexible(payment, amount_dollars):
    if isinstance(payment, PaymentMethod):
        print("Flexible handler accepts this payment object.")
        payment.process_payment(amount_dollars)
    else:
        print("Flexible handler rejects this payment object.")


base_payment = PaymentMethod()
card_payment = CreditCardPayment()


print("Using strict handler with base payment object.")
handle_payment_strict(base_payment, 50)


print("\nUsing strict handler with credit card payment.")
handle_payment_strict(card_payment, 75)


print("\nUsing flexible handler with credit card payment.")
handle_payment_flexible(card_payment, 75)



## **2. Subclass Relationships**

### **2.1. Checking subclass relations**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_06/Lecture_A/image_02_01.jpg?v=1766292090" width="250">



>* Subclass checks ask if one type specializes another
>* They show valid operations and safe expectations

>* Subclass info refines what an object is
>* It reveals shared rules plus extra specific behaviors

>* Subclass checks link different objects under one category
>* Share common rules while keeping unique identities



In [ ]:
#@title Python Code - Checking subclass relations

# Demonstrate simple subclass relationships using issubclass checks.
# Show how specialized classes relate to a common base class.
# Connect subclass checks with reasoning about object identity.

class Vehicle:
    def describe(self):
        return "Generic vehicle with unknown maximum speed."


class Car(Vehicle):
    def describe(self):
        return "Car vehicle with maximum speed around one hundred mph."


class Bus(Vehicle):
    def describe(self):
        return "Bus vehicle with many seats and slower city speed."


# Check whether Car and Bus are subclasses of Vehicle using issubclass.
print("Is Car a subclass of Vehicle?", issubclass(Car, Vehicle))


print("Is Bus a subclass of Vehicle?", issubclass(Bus, Vehicle))


# Create two different objects and show their identities and shared base.
car_one = Car()
car_two = Car()


print("car_one id and type:", id(car_one), type(car_one))


print("car_two id and type:", id(car_two), type(car_two))


print("Both share Vehicle base:", isinstance(car_one, Vehicle), isinstance(car_two, Vehicle))


print("car_one description:", car_one.describe())



### **2.2. Abstract base classes**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_06/Lecture_A/image_02_02.jpg?v=1766292134" width="250">



>* Abstract base classes describe shared roles and behaviors
>* They group different classes by capabilities, not implementation

>* ABCs group types by shared behaviors, not implementation
>* isinstance checks behavior-based roles across many classes

>* Different objects share roles despite distinct identities
>* Identity differs from behavior, enabling flexible program design



In [ ]:
#@title Python Code - Abstract base classes

# Demonstrate abstract base classes and shared behavior roles using simple examples.
# Show different concrete classes behaving like the same abstract container type.
# Connect behavior roles with identity values using id and isinstance together.

from collections.abc import Mapping, Sequence  # Import abstract base classes for mappings and sequences.

config_dict = {"width_inches": 10, "height_inches": 5}  # Regular dictionary representing some configuration values.
settings_like = type("Settings", (), {"__getitem__": lambda self, k: 42, "__iter__": lambda self: iter([]), "__len__": lambda self: 1})()  # Simple mapping like object.

numbers_list = [1, 2, 3, 4]  # Regular list behaving like a sequence container of numbers.
custom_sequence = type("FeetSequence", (), {"__len__": lambda self: 3, "__getitem__": lambda self, i: i * 3})()  # Custom sequence representing feet distances.

print("config_dict is Mapping:", isinstance(config_dict, Mapping))  # Check mapping role for regular dictionary object.
print("settings_like is Mapping:", isinstance(settings_like, Mapping))  # Check mapping role for custom mapping like object.
print("numbers_list is Sequence:", isinstance(numbers_list, Sequence))  # Check sequence role for regular list object.

print("custom_sequence is Sequence:", isinstance(custom_sequence, Sequence))  # Check sequence role for custom sequence object.
print("config_dict id:", id(config_dict))  # Show identity value for configuration dictionary object.

print("settings_like id:", id(settings_like))  # Show identity value for custom mapping like object.
print("numbers_list id:", id(numbers_list))  # Show identity value for regular list sequence object.
print("custom_sequence id:", id(custom_sequence))  # Show identity value for custom sequence object.

print("Different ids, shared roles:", isinstance(config_dict, Mapping) and isinstance(settings_like, Mapping))  # Highlight different identities but shared abstract roles.



### **2.3. Handling nonclass inputs**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_06/Lecture_A/image_02_03.jpg?v=1766292177" width="250">



>* Inputs to subclass checks aren’t always classes
>* Validate and handle bad inputs with clear errors

>* Plugin inputs may not be actual classes
>* Validate class-like inputs and give clear errors

>* Design clear, flexible handling for nonclass inputs
>* Choose strictness level to prevent subtle type bugs



In [ ]:
#@title Python Code - Handling nonclass inputs

# Demonstrate handling nonclass inputs for subclass checking safely.
# Show clear error messages when given values are not real classes.
# Compare behavior with correct classes and several incorrect input types.

from collections.abc import Sequence, Mapping, Iterable


def is_sequence_subclass(candidate):
    """Return True when candidate is subclass of Sequence, otherwise handle safely."""
    if not isinstance(candidate, type):
        print(f"Rejected input {candidate!r}, expected a class object.")
        return False
    try:
        result = issubclass(candidate, Sequence)
        print(f"Checked {candidate.__name__}, subclass of Sequence: {result}.")
        return result
    except TypeError as error:
        print(f"TypeError for {candidate!r}, message: {error}.")
        return False


class MyList(list):
    pass


def demo_inputs():
    print("Correct class input example follows below.")
    is_sequence_subclass(MyList)
    print("Incorrect instance input example follows below.")
    is_sequence_subclass(MyList())
    print("Incorrect string input example follows below.")
    is_sequence_subclass("MyList")
    print("Incorrect unrelated class input follows below.")
    is_sequence_subclass(dict)
    print("Incorrect None sentinel input follows below.")
    is_sequence_subclass(None)


demo_inputs()



## **3. Identity And Length**

### **3.1. Understanding Object Identity**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_06/Lecture_A/image_03_01.jpg?v=1766292226" width="250">



>* Object identity means a specific instance in memory
>* Same-looking objects can differ, like separate book copies

>* Identity stays while mutable objects change internally
>* Value shows current data; identity tracks instance

>* Length reflects an object's current contents, not identity
>* Identity tracks the specific container; length its state



In [ ]:
#@title Python Code - Understanding Object Identity

# Demonstrate object identity using id and equality comparisons.
# Show two lists with same contents but different identities.
# Show one list changing while its identity stays the same.

# Create two separate lists with identical contents.
list_a = ["Alice", "Bob", "Cara"]
list_b = ["Alice", "Bob", "Cara"]

# Print values and identity numbers for both lists.
print("list_a value:", list_a, "id:", id(list_a))
print("list_b value:", list_b, "id:", id(list_b))

# Compare equality of values and identity sameness.
print("Same values?", list_a == list_b)
print("Same identity?", list_a is list_b)

# Change list_a contents while keeping its identity constant.
list_a.append("Dave")
print("After append, list_a:", list_a, "id:", id(list_a))

# Show list_b remains unchanged with its own identity.
print("list_b unchanged:", list_b, "id:", id(list_b))



### **3.2. Aliasing and shared state**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_06/Lecture_A/image_03_02.jpg?v=1766292271" width="250">



>* len measures shared objects, not variable names
>* Different aliases give same length for one object

>* Mutable shared objects change through any alias
>* Think variables as labels on one shared box

>* Shared underlying data can change reported lengths
>* Stable variable names don’t guarantee stable sizes



In [ ]:
#@title Python Code - Aliasing and shared state

# Show how aliasing shares length changes between variables.
# Demonstrate that both names reference one shared list object.
# Illustrate how mutating through one alias changes observed length.

students_main = ["Ann", "Bob"]  # Create initial list with two student names.
students_alias = students_main  # Create alias name referencing the same list.
print("Initial lengths for both names:", len(students_main), len(students_alias))

print("Are both names same object:", students_main is students_alias)  # Check identity.
print("Object identity number main:", id(students_main))  # Show identity number main.
print("Object identity number alias:", id(students_alias))  # Show identity number alias.

students_main.append("Cara")  # Append new student using main variable alias.
print("After append using main, lengths:", len(students_main), len(students_alias))

students_alias.append("Dave")  # Append another student using alias variable.
print("After append using alias, lengths:", len(students_main), len(students_alias))

print("Final shared list contents:", students_main)  # Show final shared list contents.



### **3.3. Length Protocol Basics**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_06/Lecture_A/image_03_03.jpg?v=1766292318" width="250">



>* Length asks how many elements an object contains
>* Different containers share one consistent length protocol

>* Custom classes can also define meaningful lengths
>* Then code treats them like built-in containers

>* Length should consistently match current elements present
>* Design containers so length aligns with iteration



In [ ]:
#@title Python Code - Length Protocol Basics

# Demonstrate len with built-in containers and custom objects.
# Show that different containers answer the same length question.
# Connect length with iteration and stable, meaningful counts.

# Create several built-in containers with different element counts.
shopping_list = ["eggs", "milk", "bread", "butter"]
message_text = "hello world"
unique_zip_codes = {90210, 10001, 60614}

# Print their lengths using the shared len protocol function.
print("Shopping list item count:", len(shopping_list))
print("Message character count:", len(message_text))
print("Unique ZIP code count:", len(unique_zip_codes))

# Define a simple custom container class that supports the length protocol.
class Playlist:
    def __init__(self, name, songs):
        self.name = name
        self.songs = list(songs)

    def __len__(self):
        return len(self.songs)

    def __iter__(self):
        return iter(self.songs)

# Create a playlist instance and show its length behaves like built-ins.
road_trip_playlist = Playlist("Road Trip", ["Song A", "Song B", "Song C"])
print("Playlist song count:", len(road_trip_playlist))

# Modify the playlist and show the reported length changes with its state.
road_trip_playlist.songs.append("Song D")
print("Playlist song count after adding:", len(road_trip_playlist))

# Confirm that iterating over the playlist visits exactly len(playlist) songs.
visited_songs = [song for song in road_trip_playlist]
print("Visited songs equals length:", len(visited_songs) == len(road_trip_playlist))



# <font color="#418FDE" size="6.5" uppercase>**Types And Identity**</font>


In this lecture, you learned to:
- Use type, isinstance, and issubclass to inspect and reason about object types and class hierarchies. 
- Interpret id values to understand object identity and lifetime in simple scenarios. 
- Apply len to built-in containers and custom objects that implement the length protocol. 

In the next Lecture (Lecture B), we will go over 'dir Vars Help'